# Global and Regional tendencies

In this notebook I just wanted to get a view on the global tendencies. More to come as soon as I have a bit of time...

I am eager to get your comments. 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib.ticker import FuncFormatter as ff
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
%matplotlib inline
g = pd.read_csv(r'../input/global.csv')
r = pd.read_csv(r'../input/regional.csv')
g.set_index('year', inplace=True)
g = g[g.index>1945]
r.set_index('year', inplace=True) 
r = r[r.index>1945]

# Global overview
First, we look at the largest religions today:

In [ ]:
relis_num = [a for a in g.columns if (a.find('_all')!=-1) & (a.find('religion_all')!=0)]
relis_pct =[
 'christianity_percent',
 'judaism_percent',
 'islam_percent',
 'buddhism_percent',
 'zoroastrianism_percent',
 'hinduism_percent',
 'sikhism_percent',
 'shinto_percent',
 'baha’i_percent',
 'taoism_percent',
 'jainism_percent',
 'confucianism_percent',
 'syncretism_percent',
 'animism_percent',
 'otherreligion_percent', 'noreligion_percent'] 
import seaborn as sns
fig, ax = plt.subplots(figsize=(10,6.2))

current = g.index.max()
ax.set_title('largest current religions')
recent = (g[g.index==current][relis_num].T).sort_values(by=current).iloc[-6:]
recent.T.plot(kind='bar', ax=ax, legend=False)
ax.legend([i.split('_')[0] for i in recent.index])
ax.get_yaxis().set_major_formatter(ff(lambda x, p: format(int(x/1000000), ',')))
ax.set_ylabel('believers in millions')
plt.tight_layout()
plt.show()
main_relis_num = list(recent.index)
main_relis_pct = [a.split('_')[0]+'_'+'percent' for a in recent.index]

In [ ]:
fig, ax = plt.subplots(figsize=(10,6.2))
g[main_relis_pct].sum(axis=1).plot(ax=ax)
ax.get_yaxis().set_major_formatter(ff(lambda x, p: format('{:3.0%}'.format(x))))

## Time evolution of the main religions
These main religions represent the faith of the vast majority of the world population.

In [ ]:
fig, ax = plt.subplots(figsize=(10,6.2))
for i in main_relis_num: ax.plot(g[i], label=i.split('_')[0])
ax.set_title('Overview')
ax.legend([i.split('_')[0] for i in main_relis_num])
ax.get_yaxis().set_major_formatter(ff(lambda x, p: format(int(x/1000000), ',')))
ax.set_ylabel('believers in millions')
plt.tight_layout()


plt.show()

We see that there is a common trend, mainly due to the world population growth. Not also the acceleration for Islam since the mid-eighties. 

To adjust for the population growth, we look at the same but as proportion of the world population.

In [ ]:
fig, ax = plt.subplots(figsize=(10,6.2))
for i in main_relis_pct: ax.plot(g[i], label=i.split('_')[0])
ax.set_title('Overview (pct of World population)')
ax.legend([i.split('_')[0] for i in main_relis_pct])
ax.get_yaxis().set_major_formatter(ff(lambda x, p: '{:3.0%}'.format(x)))
plt.tight_layout()
plt.show()
colors = {a.split('_')[0]:'k' for a in relis_num}
for a in ax.get_lines(): colors [a.get_label()] = a.get_color() 

* Christianity is the majority with about 30% of the world population.

* There is an increase in the proportion of muslims globally.

# Religious mixes Globally and by region
We saw that Globally, there has been a big shift in the main religious mix between 1985 and 2010. Let's quantify that and see if the same shift happen in the various regions.

In [ ]:
gr = r.groupby('region')
fig, aa = plt.subplots(3,2, figsize=(10,6.2))
i=1
for re, dre in gr:
    ax = aa[np.mod(i,3), i//3]
    i += 1
    d = dre[main_relis_pct]
    d = d.apply(lambda x: x.loc[2010]-x.loc[1985])
    d.sort_values(inplace=True)
    d.plot(kind='barh', ax=ax, color=[colors[a.split('_')[0]] for a in d.index])
    #ax.set_yticklabels([str(a.get_text()).split('_')[0] for a in ax.get_yticklabels()])
    ax.set_yticklabels([])
    ax.set_title(re)
    ax.get_xaxis().set_major_formatter(ff(lambda x, p: '{:3.1%}'.format(x)))
i=0
ax = aa[np.mod(i,3), i//3]
d = g[main_relis_pct].apply(lambda x: x.loc[2010]-x.loc[1985])
d.sort_values(inplace=True)
d.plot(kind='barh', ax=ax, color=[colors[a.split('_')[0]] for a in d.index])
ax.set_yticklabels([str(a.get_text()).split('_')[0] for a in ax.get_yticklabels()])
ax.set_title('Global')
ax.get_xaxis().set_major_formatter(ff(lambda x, p: '{:3.1%}'.format(x)))
plt.tight_layout()
plt.show()

* Globally Islam and Hinduism increased to the expense of no-religious people and Christianity.
* Asia has the same pattern then the global picture except that Syncretism replaces Christianity.
* Europe witnessed a large increase in Christianity and a decrease in No religious people. Is that due to the collapse of the soviet bloc ? 
* The Western Hemisphere witnessed the opposite tendency then Europe. 
* Africa witnessed a large increase in Christianity and Islam. Is it the cause or caused by the visits of the various Popes to the continent ? 
* For Africa, the increase is not at the expense of a main global religion. We may look at this continent more specifically. 
* The middle east sow little changes; a marginal decrease in Islam and an increase in no religious people.

## A Zoom into Africa
We look at the absolute variations larger than 0.5% and see that the increase in Christianity and Islam is at the expense of Animism.


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6.2))
d = gr.get_group('Africa')[relis_pct].apply(lambda x: x.loc[2010]-x.loc[1985]).sort_values()
d = d[d.abs()>0.005]
d.plot(kind='barh', ax=ax, color=[colors[a.split('_')[0]] for a in d.index])
ax.set_yticklabels([str(a.get_text()).split('_')[0] for a in ax.get_yticklabels()])
ax.get_xaxis().set_major_formatter(ff(lambda x, p: '{:3.0%}'.format(x)))
plt.tight_layout()
plt.show()

# Contribution of the regions
Now, we do like to look at the contributions of the various regions to the change in the global religious landscape. Note that the contributions depend on three factors:
* The change in the regions landscape
* The growth of the regions population
* The size itself of the regions population

## Population size and growth

In [ ]:
popus = r.reset_index().groupby(['year', 'region']).population.sum().unstack()
fig, aa = plt.subplots(2,1, figsize=(10,12.4))
ax=aa[0]
ax.plot(popus.iloc[-6:]/1000000)
ax.plot(g.iloc[-6:].population/1000000)
ax.set_ylabel('Population in millions')
ax.legend(list(popus.columns) + ['Global'])
ax=aa[1]
ax.semilogy(popus.iloc[-6:]/popus.iloc[-6])
ax.semilogy(g.iloc[-6:].population/g.population.iloc[-6])
ax.set_ylabel('Population Growth')
plt.tight_layout()
plt.show()

Obviously, Asia is the largest region by population whose variations matter most. Africa and the Middle East are the fastest growing region whose religious mix matter more and more. 

For our period of study (1985-2010) here are the main important population figures:

In [ ]:
popus = r.reset_index().groupby(['year', 'region']).population.sum().unstack()
popus = popus[(popus.index>=1985)]
popus_mean = popus.mean(axis=0)/1000000
popus_mean['Global'] = g['population'].mean()/1000000
popus_growth = (popus.loc[2010]/popus.loc[1985])**(1/(len(popus)*5))-1
popus_growth['Global'] = (g.loc[2010,'population']/g.loc[1985,'population'])**(1/(len(g)*5))-1
fig, aa = plt.subplots(2,1, figsize=(10,12.4))
ax=aa[0]
popus_mean.sort_values().plot(kind='barh', ax=ax)
ax.set_xlabel('Mean Population in millions')
ax.legend(popus.columns)
ax=aa[1]
popus_growth.sort_values().plot(kind='barh', ax=ax)
ax.set_xlabel('Population Annual Growth')
ax.get_xaxis().set_major_formatter(ff(lambda x, p: '{:3.1%}'.format(x)))
plt.tight_layout()
plt.show()

* Asia is by far the largest region by population size.

* Africa and the middle east have fast growing populations.

* Asia, Africa and middle east are regions that matter most just because of the size and growth of their populations.

## Main religious shifts and contributing regions
We look at the contributions of the various regions to these changes.

In [ ]:
gr = r.groupby('region')
fig, aa = plt.subplots(3,2, figsize=(10,12.4))
i=1
for re, dre in gr:
    ax = aa[np.mod(i,3), i//3]
    i += 1
    d = dre[main_relis_pct].multiply(dre.worldpopulation_percent, axis=0)
    d = d.apply(lambda x: x.loc[2010]-x.loc[1985])
    d.sort_values(inplace=True)
    d.plot(kind='barh', ax=ax, color=[colors[a.split('_')[0]] for a in d.index])
    #ax.set_yticklabels([str(a.get_text()).split('_')[0] for a in ax.get_yticklabels()])
    ax.set_yticklabels([])
    ax.set_title(re)
    ax.get_xaxis().set_major_formatter(ff(lambda x, p: '{:3.1%}'.format(x)))
i=0
ax = aa[np.mod(i,3), i//3]
d = g[main_relis_pct].apply(lambda x: x.loc[2010]-x.loc[1985])
d.sort_values(inplace=True)
d.plot(kind='barh', ax=ax, color=[colors[a.split('_')[0]] for a in d.index])
ax.set_yticklabels([str(a.get_text()).split('_')[0] for a in ax.get_yticklabels()])
ax.set_title('Global')
ax.get_xaxis().set_major_formatter(ff(lambda x, p: '{:3.1%}'.format(x)))
plt.tight_layout()
plt.show()

The regional contributions are very heterogenious. Asia, because of it's size and also it's growht, is inline with the global changes. 

It is interesting to see that Europe contributed negatively to no religious people as well as christians. This is mainly due to the diminishing european population.

Africa contributed positively to christianity, componsating the decrease coming from Europe.

Does it explain or is it explained by the numerous visits of the various Popes to Africa ? 

Africa played that role because of a shift towards christianity as well as a strong population growth.

The main driver of the islamisation of the world are Asia and Africa and despite it's relative small population the Middle East. The Midlle East played that role solely thanks to its population growth.

The global increase in hinduism comes from Asia. 

# What is next ? 

Next time, I plan to analyse in further details the increase in the muslim population (Shia, Sunni, ...) 